https://www.youtube.com/watch?v=KxAzKh-xU7Y

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("Cols").master("local[*]").getOrCreate()

In [4]:
data = [['2023-01-01'], 
['2023-01-02'], 
['2023-01-03'], 
['2023-01-04'] ,
['2023-01-07'],
['2023-01-10'], 
['2023-01-11'], 
['2023-01-12'],
['2023-01-15']]

df = spark.createDataFrame(data,['Date_Given'])

In [13]:
import pyspark.sql.functions as F
df1 = df.select(F.to_date(df.Date_Given,'yyyy-MM-dd').alias('dateg'))
df1.printSchema()

root
 |-- dateg: date (nullable = true)



In [14]:
df1.show()

+----------+
|     dateg|
+----------+
|2023-01-01|
|2023-01-02|
|2023-01-03|
|2023-01-04|
|2023-01-07|
|2023-01-10|
|2023-01-11|
|2023-01-12|
|2023-01-15|
+----------+



In [15]:
#now we need to group the continuos values together, after which we can select the min and max of those continuos dates to get the start and end date

#to do that if we subtract continus dates by continous integers, we will get the same output.
# for ex if we subtract 6,7,8 by 1,2,3 we will get answer as 5 in all three case. We will use this to group continuous dates
from pyspark.sql.window import Window
window_spec = Window.orderBy('dateg')
df2 = df1.withColumn('group', df1.dateg - F.row_number().over(window_spec))
df2.show()


+----------+----------+
|     dateg|     group|
+----------+----------+
|2023-01-01|2022-12-31|
|2023-01-02|2022-12-31|
|2023-01-03|2022-12-31|
|2023-01-04|2022-12-31|
|2023-01-07|2023-01-02|
|2023-01-10|2023-01-04|
|2023-01-11|2023-01-04|
|2023-01-12|2023-01-04|
|2023-01-15|2023-01-06|
+----------+----------+



In [18]:
#after we got same output group date values for the continuos date, we will get the min and max as start and end date
df3 = df2.groupBy(df2.group).agg(F.min('dateg').alias('start_date'), F.max('dateg').alias('end_date') ).drop('group')
df3.show()

+----------+----------+
|start_date|  end_date|
+----------+----------+
|2023-01-01|2023-01-04|
|2023-01-07|2023-01-07|
|2023-01-10|2023-01-12|
|2023-01-15|2023-01-15|
+----------+----------+

